In [ ]:
pip install torch torchvision timm matplotlib numpy scikit-learn opencv-python streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 71.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 104.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6

In [ ]:
!pip install timm scikit-learn matplotlib

In [ ]:
pip install torch torchvision timm transformers scikit-learn matplotlib

In [3]:
from google.colab import files
from PIL import Image
import io

def upload_images():
    uploaded = files.upload()
    filenames = list(uploaded.keys())
    if len(filenames) != 2:
        raise Exception("Please upload exactly 2 images.")
    imgs = [Image.open(io.BytesIO(uploaded[name])).convert("L") for name in filenames]
    return imgs[0], imgs[1]

In [ ]:
# STEP 1: Install and Import Required Libraries
!pip install torchvision tqdm --quiet

import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score, precision_score
from PIL import Image
import numpy as np
from tqdm import tqdm
from google.colab import files
import io

# STEP 2: Setup Device and Transform
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

transform = transforms.Compose([
    transforms.Grayscale(3),
    transforms.Resize((96, 96)),  # smaller size = faster
    transforms.ToTensor()
])

# STEP 3: Prepare Dataset from MNIST Digits
mnist = datasets.MNIST(root=".", train=True, download=True)
genuine = [img for img in mnist if img[1] <= 4][:150]
forged = [img for img in mnist if img[1] > 4][:150]

class SignaturePairDataset(Dataset):
    def __init__(self, genuine, forged, transform):
        self.transform = transform
        self.pairs, self.labels = [], []

        for i in range(min(len(genuine) - 1, 100)):
            self.pairs.append((genuine[i][0], genuine[i + 1][0]))
            self.labels.append(1)
            self.pairs.append((genuine[i][0], forged[i][0]))
            self.labels.append(0)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        img1 = self.transform(self.pairs[idx][0])
        img2 = self.transform(self.pairs[idx][1])
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        return img1, img2, label

dataset = SignaturePairDataset(genuine, forged, transform)
train_size = int(0.8 * len(dataset))
train_set, test_set = torch.utils.data.random_split(dataset, [train_size, len(dataset) - train_size])
train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
test_loader = DataLoader(test_set, batch_size=8)

# STEP 4: Define Light Siamese Model using ResNet18
class SiameseResNet(nn.Module):
    def __init__(self):
        super().__init__()
        resnet = models.resnet18(pretrained=True)
        resnet.fc = nn.Identity()
        self.backbone = resnet
        self.fc = nn.Sequential(
            nn.Linear(512, 128),
            nn.ReLU(),
            nn.Linear(128, 1)  # raw logit output
        )

    def forward_once(self, x):
        return self.backbone(x)

    def forward(self, x1, x2):
        f1 = self.forward_once(x1)
        f2 = self.forward_once(x2)
        diff = torch.abs(f1 - f2)
        return self.fc(diff).squeeze()

# STEP 5: Train Model
model = SiameseResNet().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

def train(model, loader, epochs=10):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        for img1, img2, label in tqdm(loader, desc=f"Epoch {epoch+1}"):
            img1, img2, label = img1.to(device), img2.to(device), label.to(device)
            pred = model(img1, img2)
            loss = criterion(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        print(f" Epoch {epoch+1} - Loss: {total_loss:.4f}")

train(model, train_loader)

# STEP 6: Evaluate Model
def evaluate(model, loader):
    model.eval()
    preds, labels = [], []
    with torch.no_grad():
            for i in range(5):
                img1, img2, label = test_set[i]
                pred = model(img1.unsqueeze(0).to(device), img2.unsqueeze(0).to(device)).item()
                prob = torch.sigmoid(torch.tensor(pred)).item()
                print(f"GT: {label}, Logit: {pred:.4f}, Probability: {prob:.4f}")
    acc = accuracy_score(labels, preds)
    prec = precision_score(labels, preds)
    print(f"\n Accuracy: {acc * 100:.2f}%")
    print(f" Precision: {prec * 100:.2f}%")


evaluate(model, test_loader)

#  STEP 7: Upload and Verify Signatures
def upload_images():
    print(" Upload original signature image")
    uploaded = files.upload()
    original_path = list(uploaded.keys())[0]

    print("\n  Upload test signature image")
    uploaded = files.upload()
    test_path = list(uploaded.keys())[0]

    return original_path, test_path

def verify_signature(model, original_path, test_path):
    model.eval()
    img1 = Image.open(original_path).convert("L")
    img2 = Image.open(test_path).convert("L")
    img1 = transform(img1).unsqueeze(0).to(device)
    img2 = transform(img2).unsqueeze(0).to(device)
    with torch.no_grad():
        logit = model(img1, img2).item()
        prob = torch.sigmoid(torch.tensor(logit)).item()
        print(f"\n🔎 Similarity Score: {prob:.4f}")
        if prob > 0.5:
            print("Signature Match: Genuine")
        else:
            print("Signature Mismatch: Forged")

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1: 100%|██████████| 20/20 [00:19<00:00,  1.01it/s]


✅ Epoch 1 - Loss: 13.6793


Epoch 2: 100%|██████████| 20/20 [00:20<00:00,  1.04s/it]


✅ Epoch 2 - Loss: 11.0797


Epoch 3: 100%|██████████| 20/20 [00:19<00:00,  1.02it/s]


...

Epoch 9: 100%|██████████| 20/20 [00:20<00:00,  1.02s/it]


✅ Epoch 9 - Loss: 2.5019


Epoch 10: 100%|██████████| 20/20 [00:19<00:00,  1.03it/s]


✅ Epoch 10 - Loss: 1.9404
GT: 0.0, Logit: 2.7401, Probability: 0.9393
GT: 1.0, Logit: 0.5780, Probability: 0.6406
GT: 0.0, Logit: -6.1083, Probability: 0.0022
GT: 0.0, Logit: -6.4985, Probability: 0.0015
GT: 1.0, Logit: -3.3031, Probability: 0.0355

📊 Accuracy: nan%
📊 Precision: 0.00%


/usr/local/lib/python3.11/dist-packages/numpy/lib/_function_base_impl.py:557: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [4]:
def verify_signature(model, original_img, test_img):
    model.eval()
    img1 = transform(original_img).unsqueeze(0).to(device)
    img2 = transform(test_img).unsqueeze(0).to(device)
    with torch.no_grad():
        score = model(img1, img2).item()
        print(f"\n🧠 Similarity Score: {score:.4f}")
        if score > 0.5:
            print("✅ Signature Match: Likely Genuine")
        else:
            print("❌ Signature Mismatch: Likely Forged")

In [10]:
pip install torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 918.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitli

In [12]:
# Install required packages
!pip install torch torchvision

# 1. Imports
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from sklearn.metrics import accuracy_score, precision_score
import numpy as np
import random
from google.colab import files
import io
import os

# 2. Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 3. Simple Siamese Network for demonstration
class SiameseNetwork(nn.Module):
    def __init__(self):
        super(SiameseNetwork, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 5 * 5, 256),
            nn.ReLU(),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )

    def forward_once(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

    def forward(self, input1, input2):
        out1 = self.forward_once(input1)
        out2 = self.forward_once(input2)
        return torch.abs(out1 - out2)

# 4. Dummy dataset for quick training
class DummySignatureDataset(Dataset):
    def __init__(self, num_pairs=1000, transform=None):
        self.transform = transform
        self.num_pairs = num_pairs

    def __getitem__(self, idx):
        img1 = Image.fromarray((np.random.rand(28, 28) * 255).astype('uint8')).convert("L")
        img2 = Image.fromarray((np.random.rand(28, 28) * 255).astype('uint8')).convert("L")
        label = random.randint(0, 1)
        if label == 0:  # same
            img2 = img1.copy()
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.tensor([label], dtype=torch.float32)

    def __len__(self):
        return self.num_pairs

transform = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.ToTensor()
])

train_loader = DataLoader(DummySignatureDataset(transform=transform), batch_size=32, shuffle=True)

# 5. Train model quickly (small dummy run)
model = SiameseNetwork().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Training on dummy data...")
model.train()
for epoch in range(2):  # quick training for demo
    for img1, img2, labels in train_loader:
        img1, img2, labels = img1.to(device), img2.to(device), labels.to(device)
        outputs = model(img1, img2)
        loss = criterion(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1} Loss: {loss.item():.4f}")

torch.save(model.state_dict(), "signature_verifier.pth")
print("✅ Model trained and saved.")

# 6. Upload function
def upload_images():
    uploaded = files.upload()
    filenames = list(uploaded.keys())
    if len(filenames) != 2:
        raise Exception("Please upload exactly 2 images.")
    imgs = [Image.open(io.BytesIO(uploaded[name])).convert("L") for name in filenames]
    return imgs[0], imgs[1]

# 7. Verification function
def verify_signature(model, original_img, test_img):
    model.eval()
    img1 = transform(original_img).unsqueeze(0).to(device)
    img2 = transform(test_img).unsqueeze(0).to(device)
    with torch.no_grad():
        score = model(img1, img2).item()
        print(f"\n🧠 Similarity Score: {score:.4f}")
        if score > 0.5:
            print("✅ Signature Match: Likely Genuine")
        else:
            print("❌ Signature Mismatch: Likely Forged")

# 8. Run verification
print("\n📤 Upload 2 images to verify...")
original, test = upload_images()
verify_signature(model, original, test)


Training on dummy data...
Epoch 1 Loss: 2.0145
Epoch 2 Loss: 2.7422
✅ Model trained and saved.

📤 Upload 2 images to verify...


Saving orginal .jpg to orginal  (1).jpg
Saving duplicate sign.jpg to duplicate sign (1).jpg

🧠 Similarity Score: 0.0004
❌ Signature Mismatch: Likely Forged
